In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch import nn

import wandb

print(torch.__version__)
print(torch.backends.mps.is_available())

1.12.1
True


In [3]:
df = pd.read_csv('Data/cleaned.csv', index_col=0)
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0
764,2,122.0,70.0,27.0,102.5,36.8,0.340,27,0
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0
766,1,126.0,60.0,32.0,169.5,30.1,0.349,47,1


In [ ]:
def to_tensor(X_train, X_test, y_train, y_test):
    X_train = torch.from_numpy(X_train.values).float()
    X_test = torch.from_numpy(X_test.values).float()
    y_train = torch.from_numpy(y_train.values).float()
    y_test = torch.from_numpy(y_test.values).float()

    return X_train, X_test, y_train, y_test


X, y = df.copy().drop('Outcome', axis=1), df.Outcome
X_train, X_test, y_train, y_test = to_tensor(*train_test_split(X, y, test_size=0.3))
print(X_train, y_train)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to('mps')
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters())

In [ ]:
def train(X, y, model, loss_fn, optimizer, i):
    model.train()

    X, y = X.to('mps'), y.to('mps')

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        print(f"loss: {loss.item():>7f}")


def test(X, y, model, loss_fn, i):
    model.eval()

    test_loss, correct = 0, 0

    X, y = X.to('mps'), y.to('mps')
    pred = model(X)
    test_loss += loss_fn(pred, y).item()
    correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    correct /= X.shape[0]

    if i % 100:
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


epochs = 1000
for t in range(epochs):
    if t % 100:
        print(f"Epoch {t+1}\n-------------------------------")

    train(X_train, y_train, model, loss_fn, optimizer, t)
    test(X_test, y_test, model, loss_fn, t)

print("Done!")